# Notes
- this worked well for the planned network additions
- did not work for the beehive bikeways because they weren't snapped to the MMN

In [1]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [2]:
# # fill NA values in Spatially enabled dataframes (ignores SHAPE column)
# def fill_na_sedf(df_with_shape_column, fill_value=0):
#     if 'SHAPE' in list(df_with_shape_column.columns):
#         df = df_with_shape_column.copy()
#         shape_column = df['SHAPE'].copy()
#         del df['SHAPE']
#         return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
#     else:
#         raise Exception("Dataframe does not include 'SHAPE' column")

# Setup Outputs directories

In [2]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "bb.gdb", 'planned.gdb', 'bb_planned.gdb', 'mmn_beehive_modeling.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])
gdb3 = os.path.join(outputs[0], outputs[3])
gdb4 = os.path.join(outputs[0], outputs[4])


if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

if not arcpy.Exists(gdb3):
    arcpy.CreateFileGDB_management(outputs[0], outputs[3])

if not arcpy.Exists(gdb4):
    arcpy.CreateFileGDB_management(outputs[0], outputs[4])


## Add BB to MMN

In [4]:
mmn = r'E:\Tasks\Bike_Modeling_Data_Update\MM_NetworkDataset_08132024.gdb\NetworkDataset\BikePedAuto'
beehive_bikeways = r'E:\Tasks\Bike_Modeling_Data_Update\Beehive_Bikeways_20230822.shp'
above_grade_intersections = r'.\Inputs\Above_Grade_Intersections.shp'

# create a working version of the MMN and add unique ID
mmn_copy = arcpy.management.CopyFeatures(mmn, os.path.join(gdb, '_00_mmn_copy'))
mmn_lyr = arcpy.management.MakeFeatureLayer(mmn_copy, "mmn_lyr")


# select highways that cross BB at-grade
arcpy.SelectLayerByLocation_management(mmn_lyr, 'INTERSECT', above_grade_intersections, selection_type='NEW_SELECTION')
arcpy.SelectLayerByAttribute_management(mmn_lyr, "SWITCH_SELECTION")

# split the BB layer at same junctions as MMN
mmn_vertices = arcpy.management.FeatureVerticesToPoints(mmn_lyr, os.path.join(gdb, '_02_mmn_vertices'), "BOTH_ENDS")
bb_split = arcpy.management.SplitLineAtPoint(beehive_bikeways, mmn_vertices, os.path.join(gdb, '_03_bb_split'), "10 Feet")
arcpy.CalculateGeometryAttributes_management(os.path.realpath(bb_split[0]), [['Length_Miles','LENGTH_GEODESIC']],  length_unit='MILES_US')
arcpy.AddField_management(bb_split, field_name='SPLIT_ID', field_type='LONG')
arcpy.CalculateField_management(bb_split,"SPLIT_ID",'!{}!'.format('OBJECTID'))
bb_split_df = pd.DataFrame.spatial.from_featureclass(bb_split[0])

In [5]:
# select and buffer bikeways
bb_split_buffered = arcpy.analysis.Buffer(bb_split, os.path.join(gdb, '_04_bb_buffer'), "10 Feet", "FULL", "ROUND", None, None, "GEODESIC")
bb_buffered_dissolved = arcpy.analysis.Buffer(beehive_bikeways, os.path.join(gdb, '_05_bb_buffer_dissolved'), "10 Feet", "FULL", "ROUND", 'ALL', None, "GEODESIC")

# select MMN links that are within the BB buffer, delete
mmn_lyr = arcpy.management.MakeFeatureLayer(mmn_copy, "mmn_lyr2")
print(arcpy.management.GetCount(mmn_lyr))
arcpy.management.SelectLayerByLocation(
    in_layer=mmn_lyr,
    overlap_type="WITHIN",
    select_features=bb_buffered_dissolved,
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)
print(arcpy.management.GetCount(mmn_lyr))
arcpy.management.CopyFeatures(mmn_lyr, os.path.join(gdb, '_05_mmn_deleted'))
mmn_lyr_df = pd.DataFrame.spatial.from_featureclass(mmn_lyr[0])



# spatial join buffered bb links to overlapping mmn links, read into df
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(bb_split_buffered)
fieldmappings.addTable(mmn_copy)

mmn_columns = ['Name', 'Oneway', 'Speed', 'AutoNetwork', 'BikeNetwork',
       'PedNetwork', 'SourceData', 'DriveTime', 'BikeTime', 'PedestrianTime',
       'Length_Miles', 'ConnectorNetwork', 'CartoCode', 'AADT', 'AADT_YR',
       'BIKE_L', 'BIKE_R', 'VERT_LEVEL']

for col in mmn_columns:
    fieldindex = fieldmappings.findFieldMapIndex(col)
    fieldmap = fieldmappings.getFieldMap(fieldindex)
    fieldmap.mergeRule = 'First'
    fieldmappings.replaceFieldMap(fieldindex, fieldmap)

bb_mmn_sj = arcpy.SpatialJoin_analysis(bb_split_buffered, mmn_lyr, os.path.join(gdb, '_06_mmn_non_pp_join_with_bb'),'JOIN_ONE_TO_MANY', "KEEP_ALL", fieldmappings, 'CONTAINS') # OR WITHIN

arcpy.SelectLayerByAttribute_management(mmn_lyr, "SWITCH_SELECTION")
mmn_removed =arcpy.management.CopyFeatures(mmn_lyr, os.path.join(gdb, '_07_mmn_remaining'))
arcpy.CalculateGeometryAttributes_management(os.path.realpath(mmn_removed[0]), [['Length_Miles','LENGTH_GEODESIC']],  length_unit='MILES_US')

# read spatial join result into pandas and update bike feature attributes
bb_mmn_sj_df = pd.DataFrame.spatial.from_featureclass(bb_mmn_sj[0])
bb_mmn_sj_df['BIKE_L'] = bb_mmn_sj_df['TypeCode']
bb_mmn_sj_df['BIKE_R'] = bb_mmn_sj_df['TypeCode']
bb_mmn_sj_df['BB'] = pd.Series(dtype='int8')
bb_mmn_sj_df['BB'] = "1"


# join sj features to original overlapping mmn links
new_mmn_links = bb_split_df[['SPLIT_ID' ,'SHAPE']].merge(bb_mmn_sj_df.drop('SHAPE', axis=1), on='SPLIT_ID', how='left') # need original mmn attributes + 'TypeCode' aka new bike infrastructure
new_mmn_links = new_mmn_links[mmn_columns + ['BB','SHAPE']]
new_mmn_links.spatial.to_featureclass(location=os.path.join(gdb, '_08_new_mmn_features'),sanitize_columns=False)  # merge these after schema formatting

# merge MMN (with deleted links) with new links (with)
mmn_df = pd.DataFrame.spatial.from_featureclass(mmn_removed[0])
mmn_df_with_new_links = pd.concat([mmn_df, new_mmn_links])
mmn_df_with_new_links = mmn_df_with_new_links.reset_index().drop(['index','OBJECTID'], axis=1)

mmn_df_with_new_links.loc[mmn_df_with_new_links['Speed'].isna() == False, 'DriveTime'] = (mmn_df_with_new_links['Length_Miles'] / mmn_df_with_new_links['Speed']) * 60
mmn_df_with_new_links.loc[mmn_df_with_new_links['Length_Miles'].isna() == False, 'PedestrianTime'] = (mmn_df_with_new_links['Length_Miles'] / 3.1) * 60
mmn_df_with_new_links.loc[mmn_df_with_new_links['Length_Miles'].isna() == False, 'BikeTime'] = (mmn_df_with_new_links['Length_Miles'] / 9.6) * 60
mmn_df_with_new_links.loc[(mmn_df_with_new_links['BIKE_L'].isna() == False) | 
                          (mmn_df_with_new_links['BIKE_R'].isna() != False) | 
                          (mmn_df_with_new_links['CartoCode'] == '11 Other Local, Neighborhood, Rural Roads'), 
                          'BikeTime'] = mmn_df_with_new_links['Length_Miles'] / 11 * 60


final_mmn = os.path.join(gdb, '_09_mmn_with_beehive_bikeways')
mmn_df_with_new_links.spatial.to_featureclass(location=final_mmn,sanitize_columns=False)
mmn_df_with_new_links.spatial.to_featureclass(location=os.path.join(gdb4, 'mmn_with_beehive_bikeways'),sanitize_columns=False)

177178
2761


'e:\\Tasks\\Bike_Modeling_Data_Update\\Processing\\Outputs\\mmn_beehive_modeling.gdb\\mmn_with_beehive_bikeways'

# Add Planned Features to MMN

In [3]:
# roads
roads = r"E:\Tasks\Bike_Modeling_Data_Update\sgid_roads_09022024.gdb\Roads"
q1 = '''(BIKE_PLN_L IN ('1', '1B', '1C', '1D', '2A', '2B', '3', '3A', '3B', '3C', 'PP')) or (BIKE_PLN_R IN ('1', '1B', '1C', '1D', '2A', '2B', '3', '3A', '3B', '3C', 'PP'))'''
roads_lyr = arcpy.MakeFeatureLayer_management(roads, 'roads_lyr', where_clause=q1)

#trails (some of these are in BB)
trails = r"E:\Tasks\Bike_Modeling_Data_Update\Trails.shp"
q2 = """status IN ('Future', 'PROPOSED')"""
trails_lyr = arcpy.MakeFeatureLayer_management(trails, 'trails_lyr', where_clause=q2)

roads_trails = arcpy.Merge_management([roads_lyr, trails_lyr], os.path.join(gdb2, '_00_roads_trails'),add_source='ADD_SOURCE_INFO')
arcpy.CalculateGeometryAttributes_management(os.path.realpath(roads_trails[0]), [['Length_Miles','LENGTH_GEODESIC']],  length_unit='MILES_US')
roads_trails_df = pd.DataFrame.spatial.from_featureclass(roads_trails[0])[['FULLNAME' ,'CARTOCODE', 'ONEWAY', 'SPEED_LMT', 'BIKE_PLN_L', 'BIKE_PLN_R', 'MERGE_SRC', 'DOT_AADT', 'VERT_LEVEL', 'Length_Miles','SHAPE']]

roads_trails_df['Speed'] = roads_trails_df['SPEED_LMT']
roads_trails_df['ConnectorNetwork'] = 'N'

roads_trails_df['PedNetwork'] = 'Y'
roads_trails_df['BikeNetwork'] = 'Y'
roads_trails_df['AutoNetwork'] = 'Y'
roads_trails_df.loc[roads_trails_df['MERGE_SRC'] == 'trails_lyr', 'AutoNetork'] = 'N'

roads_trails_df['Name'] = roads_trails_df['FULLNAME']
roads_trails_df['Oneway'] = roads_trails_df['ONEWAY']
roads_trails_df['CartoCode'] = roads_trails_df['CARTOCODE']
roads_trails_df['AADT'] = roads_trails_df['DOT_AADT']

roads_trails_df['BIKE_L'] = roads_trails_df['BIKE_PLN_L']
roads_trails_df['BIKE_R'] = roads_trails_df['BIKE_PLN_R']

roads_trails_df['PLANNED'] = "1"

roads_trails_df.loc[roads_trails_df['Speed'].isna() == False, 'DriveTime'] = (roads_trails_df['Length_Miles'] / roads_trails_df['Speed']) * 60
roads_trails_df.loc[roads_trails_df['Length_Miles'].isna() == False, 'PedestrianTime'] = (roads_trails_df['Length_Miles'] / 3.1) * 60
roads_trails_df.loc[roads_trails_df['Length_Miles'].isna() == False, 'BikeTime'] = (roads_trails_df['Length_Miles'] / 9.6) * 60

roads_trails_df.loc[roads_trails_df['MERGE_SRC'] == 'roads_lyr', 'SourceData'] = 'RoadCenterlines'
roads_trails_df.loc[roads_trails_df['MERGE_SRC'] == 'trails_lyr', 'SourceData'] = 'Trails'

roads_trails_df = roads_trails_df[['Name','Oneway','Speed','AutoNetwork', 'BikeNetwork','PedNetwork','SourceData','DriveTime','PedestrianTime','BikeTime', 'Length_Miles','ConnectorNetwork','CartoCode','AADT','BIKE_L','BIKE_R', 'VERT_LEVEL', 'PLANNED', 'SHAPE']]

planned_features = os.path.join(gdb2, '_01_planned_features')
roads_trails_df.spatial.to_featureclass(location=planned_features,sanitize_columns=False)

C:\Users\jreynolds\AppData\Local\Temp\ipykernel_27676\2329096483.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'N' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  roads_trails_df.loc[roads_trails_df['MERGE_SRC'] == 'trails_lyr', 'AutoNetork'] = 'N'
C:\Users\jreynolds\AppData\Local\Temp\ipykernel_27676\2329096483.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'RoadCenterlines' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  roads_trails_df.loc[roads_trails_df['MERGE_SRC'] == 'roads_lyr', 'SourceData'] = 'RoadCenterlines'


'e:\\Tasks\\Bike_Modeling_Data_Update\\Processing\\Outputs\\planned.gdb\\_01_planned_features'

In [4]:
# create a working version of the MMN and add unique ID
mmn = r'E:\Tasks\Bike_Modeling_Data_Update\MM_NetworkDataset_08132024.gdb\NetworkDataset\BikePedAuto'
mmn_copy = arcpy.management.CopyFeatures(mmn, os.path.join(gdb2, '_02_mmn_copy'))
mmn_lyr = arcpy.management.MakeFeatureLayer(mmn_copy, "mmn_lyr")

# select and buffer bikeways
planned_buffered = arcpy.analysis.Buffer(planned_features, os.path.join(gdb2, '_03_planned_buffer'), "10 Feet", "FULL", "ROUND", None, None, "GEODESIC")
planned_buffered_dissolved = arcpy.analysis.Buffer(planned_features, os.path.join(gdb2, '_04_planned_buffer_dissolved'), "10 Feet", "FULL", "ROUND", 'ALL', None, "GEODESIC")

# select MMN links that are within the Planned buffer, delete
mmn_lyr = arcpy.management.MakeFeatureLayer(mmn_copy, "mmn_lyr2")
print(arcpy.management.GetCount(mmn_lyr))
# arcpy.management.SelectLayerByLocation(mmn_lyr, 'WITHIN', bb_buffered, selection_type='NEW_SELECTION')
arcpy.management.SelectLayerByLocation(
    in_layer=mmn_lyr,
    overlap_type="WITHIN",
    select_features=planned_buffered_dissolved,
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)
print(arcpy.management.GetCount(mmn_lyr))
arcpy.management.CopyFeatures(mmn_lyr, os.path.join(gdb2, '_04_mmn_deleted'))
# mmn_lyr_df = pd.DataFrame.spatial.from_featureclass(mmn_lyr[0])

arcpy.SelectLayerByAttribute_management(mmn_lyr, "SWITCH_SELECTION")
mmn_removed =arcpy.management.CopyFeatures(mmn_lyr, os.path.join(gdb2, '_05_mmn_remaining'))

mmn_with_planned_features = arcpy.Merge_management([mmn_removed, planned_features], os.path.join(gdb2, '_06_mmn_with_planned_features'))
arcpy.Merge_management([mmn_removed, planned_features], os.path.join(gdb4, 'mmn_with_planned_features'))

177178
17318


<Result '.\\Outputs\\mmn_beehive_modeling.gdb\\mmn_with_planned_features'>

# Add BB and Planned to MMN

In [5]:
mmn = mmn_with_planned_features
beehive_bikeways = r'E:\Tasks\Bike_Modeling_Data_Update\Beehive_Bikeways_20230822.shp'
above_grade_intersections = r'.\Inputs\Above_Grade_Intersections.shp'

# create a working version of the MMN and add unique ID
mmn_copy = arcpy.management.CopyFeatures(mmn, os.path.join(gdb3, '_00_mmn_copy'))
mmn_lyr = arcpy.management.MakeFeatureLayer(mmn_copy, "mmn_lyr")


# select highways that cross BB at-grade
arcpy.SelectLayerByLocation_management(mmn_lyr, 'INTERSECT', above_grade_intersections, selection_type='NEW_SELECTION')
arcpy.SelectLayerByAttribute_management(mmn_lyr, "SWITCH_SELECTION")

# split the BB layer at same junctions as MMN
mmn_vertices = arcpy.management.FeatureVerticesToPoints(mmn_lyr, os.path.join(gdb3, '_02_mmn_vertices'), "BOTH_ENDS")
bb_split = arcpy.management.SplitLineAtPoint(beehive_bikeways, mmn_vertices, os.path.join(gdb3, '_03_bb_split'), "10 Feet")
arcpy.CalculateGeometryAttributes_management(os.path.realpath(bb_split[0]), [['Length_Miles','LENGTH_GEODESIC']],  length_unit='MILES_US')
arcpy.AddField_management(bb_split, field_name='SPLIT_ID', field_type='LONG')
arcpy.CalculateField_management(bb_split,"SPLIT_ID",'!{}!'.format('OBJECTID'))
bb_split_df = pd.DataFrame.spatial.from_featureclass(bb_split[0])

In [6]:
# select and buffer bikeways
bb_split_buffered = arcpy.analysis.Buffer(bb_split, os.path.join(gdb3, '_04_bb_buffer'), "10 Feet", "FULL", "ROUND", None, None, "GEODESIC")
bb_buffered_dissolved = arcpy.analysis.Buffer(beehive_bikeways, os.path.join(gdb3, '_05_bb_buffer_dissolved'), "10 Feet", "FULL", "ROUND", 'ALL', None, "GEODESIC")

# select MMN links that are within the BB buffer, delete
mmn_lyr = arcpy.management.MakeFeatureLayer(mmn_copy, "mmn_lyr2")
print(arcpy.management.GetCount(mmn_lyr))
arcpy.management.SelectLayerByLocation(
    in_layer=mmn_lyr,
    overlap_type="WITHIN",
    select_features=bb_buffered_dissolved,
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)
print(arcpy.management.GetCount(mmn_lyr))
arcpy.management.CopyFeatures(mmn_lyr, os.path.join(gdb3, '_05_mmn_deleted'))
mmn_lyr_df = pd.DataFrame.spatial.from_featureclass(mmn_lyr[0])



# spatial join buffered bb links to overlapping mmn links, read into df
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(bb_split_buffered)
fieldmappings.addTable(mmn_copy)

mmn_columns = ['Name', 'Oneway', 'Speed', 'AutoNetwork', 'BikeNetwork',
       'PedNetwork', 'SourceData', 'DriveTime', 'BikeTime', 'PedestrianTime',
       'Length_Miles', 'ConnectorNetwork', 'CartoCode', 'AADT', 'AADT_YR',
       'BIKE_L', 'BIKE_R', 'VERT_LEVEL', 'PLANNED']

for col in mmn_columns:
    fieldindex = fieldmappings.findFieldMapIndex(col)
    fieldmap = fieldmappings.getFieldMap(fieldindex)
    fieldmap.mergeRule = 'First'
    fieldmappings.replaceFieldMap(fieldindex, fieldmap)

bb_mmn_sj = arcpy.SpatialJoin_analysis(bb_split_buffered, mmn_lyr, os.path.join(gdb3, '_06_mmn_non_pp_join_with_bb'),'JOIN_ONE_TO_MANY', "KEEP_ALL", fieldmappings, 'CONTAINS') # OR WITHIN

arcpy.SelectLayerByAttribute_management(mmn_lyr, "SWITCH_SELECTION")
mmn_removed =arcpy.management.CopyFeatures(mmn_lyr, os.path.join(gdb3, '_07_mmn_remaining'))
arcpy.CalculateGeometryAttributes_management(os.path.realpath(mmn_removed[0]), [['Length_Miles','LENGTH_GEODESIC']],  length_unit='MILES_US')

# read spatial join result into pandas and update bike feature attributes
bb_mmn_sj_df = pd.DataFrame.spatial.from_featureclass(bb_mmn_sj[0])
bb_mmn_sj_df['BIKE_L'] = bb_mmn_sj_df['TypeCode']
bb_mmn_sj_df['BIKE_R'] = bb_mmn_sj_df['TypeCode']
bb_mmn_sj_df['BB'] = "1"


# join sj features to original overlapping mmn links
new_mmn_links = bb_split_df[['SPLIT_ID' ,'SHAPE']].merge(bb_mmn_sj_df.drop('SHAPE', axis=1), on='SPLIT_ID', how='left') # need original mmn attributes + 'TypeCode' aka new bike infrastructure
new_mmn_links = new_mmn_links[mmn_columns + ['BB','SHAPE']]
new_mmn_links.spatial.to_featureclass(location=os.path.join(gdb3, '_08_new_mmn_features'),sanitize_columns=False)  # merge these after schema formatting

# merge MMN (with deleted links) with new links (with)
mmn_df = pd.DataFrame.spatial.from_featureclass(mmn_removed[0])
mmn_df_with_new_links = pd.concat([mmn_df, new_mmn_links])
mmn_df_with_new_links = mmn_df_with_new_links.reset_index().drop(['index','OBJECTID'], axis=1)

mmn_df_with_new_links.loc[mmn_df_with_new_links['Speed'].isna() == False, 'DriveTime'] = (mmn_df_with_new_links['Length_Miles'] / mmn_df_with_new_links['Speed']) * 60
mmn_df_with_new_links.loc[mmn_df_with_new_links['Length_Miles'].isna() == False, 'PedestrianTime'] = (mmn_df_with_new_links['Length_Miles'] / 3.1) * 60
mmn_df_with_new_links.loc[mmn_df_with_new_links['Length_Miles'].isna() == False, 'BikeTime'] = (mmn_df_with_new_links['Length_Miles'] / 9.6) * 60
mmn_df_with_new_links.loc[(mmn_df_with_new_links['BIKE_L'].isna() == False) | 
                          (mmn_df_with_new_links['BIKE_R'].isna() != False) | 
                          (mmn_df_with_new_links['CartoCode'] == '11 Other Local, Neighborhood, Rural Roads'), 
                          'BikeTime'] = mmn_df_with_new_links['Length_Miles'] / 11 * 60


final_mmn = os.path.join(gdb3, '_09_mmn_with_bb_and_planned')
mmn_df_with_new_links.spatial.to_featureclass(location=final_mmn,sanitize_columns=False)
mmn_df_with_new_links.spatial.to_featureclass(location=os.path.join(gdb4, 'mmn_with_bb_and_planned'), sanitize_columns=False)

176891
2633


c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  

'e:\\Tasks\\Bike_Modeling_Data_Update\\Processing\\Outputs\\mmn_beehive_modeling.gdb\\mmn_with_bb_and_planned'

## fix microzone output

In [2]:
d = pd.read_csv(r"E:\Projects\utah_bike_demand_model_2024_Planned\Create_Microzones\Outputs\microzones.csv").astype('Float64')
g =  pd.DataFrame.spatial.from_featureclass(r"E:\Tasks\MICROZONES\Inputs\Microzones_20240815.shp")
gd = g[['SHAPE','zone_id']].merge(d, on='zone_id', how='left')
gd.spatial.to_featureclass(location='microzones.shp',sanitize_columns=False) 

'e:\\Tasks\\Bike_Modeling_Data_Update\\Processing\\microzones.shp'